In [ ]:
import os
from moviepy.editor import *
import moviepy.audio.fx.all as afx


class LegoCons:
    def __init__(self,pth,consName):
        self.pth=pth
        self.consName=consName
        self.creatItem()
        
    def creatItem(self):
        New=os.path.join(self.pth,self.consName)
        if not os.path.exists(New):
            print('新项目，创建文件夹...\n')
            os.makedirs(New)
            new_totals=os.path.join(self.pth,self.consName,'零件总图')
            new_video=os.path.join(self.pth,self.consName,'video')
            print('文件夹{}创建完成\n'.format(' '+self.consName+' '))
            
            if not os.path.exists(new_totals):
                os.makedirs(new_totals)
                print('文件夹{}创建完成\n'.format(' 零件总图 '))
            if not os.path.exists(new_video):
                os.makedirs(new_video)
                print('文件夹{}创建完成\n'.format(' video '))
                
            
        else:
            print('已有项目\n')
            new_totals=os.path.join(self.pth,self.consName,'零件总图')
            new_video=os.path.join(self.pth,self.consName,'video')
            if not os.path.exists(new_totals):
                os.makedirs(new_totals)
                print('文件夹{}创建完成\n'.format(' 零件总图 '))
            if not os.path.exists(new_video):
                os.makedirs(new_video)
                print('文件夹{}创建完成\n'.format(' video '))
          
    def expHTML_lego(self):
        lsts=self.makeList()     
        
        html_pic_addr=''
        _pre_pic='''
        </section>
                        <section style="color: #bfbfbf;padding-top: 10px;padding-bottom: 10px;display: inline-block;width: 100%;box-sizing: border-box;" data-width="75%">
                            <section class="_135editor" data-tools="135编辑器" data-id="95138">
                                <section class="_135editor">
                                    <section style="margin: 1em auto;text-align: center;padding: 5px;border-width: 1px;border-style: solid;border-color: transparent;overflow: hidden;box-sizing: border-box;">
                                        <section class="135brush" data-style="display: inline-block;width: 100%;margin:0;padding:0;" style="white-space: nowrap;overflow-x: scroll;">
        '''


        pre_pic_addr='''
         <img class="" data-ratio="0.75" data-type="jpeg" data-w="1200" data-width="100%" src="https://chuntianhuahua-1257410889.cos.ap-guangzhou.myqcloud.com/legoCons/
         '''
        after_pic_addr='"/>'
        
        for lst in lsts:
            html_pic_addr=html_pic_addr+pre_pic_addr.strip()+lst+after_pic_addr+'\n'
            
        out=_pre_pic+html_pic_addr
        print('生成html...完成\n')
        
        with open(os.path.join(self.pth,self.consName,self.consName+'_html.txt'),'w') as f:
            f.write(out)
        
        print('写入txt...完成\n')
      
    def makeList(self):
        n=0
        pngList=[]
        for file in os.listdir(os.path.join(self.pth,self.consName)):
            if file[-3:].lower()=='png' and len(file)<10:
                os.rename(os.path.join(self.pth,self.consName,file),os.path.join(self.pth,self.consName,file.zfill(10)))
                pngList.append(file.zfill(10))
                n+=1
            elif file[-3:].lower()=='png':
                pngList.append(self.consName+"/"+file)
        pngList.sort()
        if n>0:
            print('改名{}个文件'.format(n))
        
        if os.path.exists(os.path.join(self.pth,self.consName,'零件总图')):
#             print('存在零件总图\n')
            totalList=[]
            for file in os.listdir(os.path.join(self.pth,self.consName,'零件总图')):
                totalList.append(self.consName+'/零件总图/'+file)
        else:
            pass
#             print('无零件总图')
        totalList.sort()
        
        outList=[self.consName+'/'+self.consName[3:]+'.jpg']
        outList.extend(totalList)
        outList.extend(pngList)
        
        print('生成 文件列表...完成\n')
        return outList    
    
class ConsMovie:
    def __init__(self,pth,consName):
        self.bgm='I:\大智小超\公共素材\声音类\legoBGM.mp3'
        self.endV='I:\大智小超\公共素材\视频类\片尾01.mp4'
        self.pth=pth
        self.consName=consName

        a=LegoCons(self.pth,self.consName)
        self.lst=a.makeList()
        
    
    def expMovie(self):
        print('正在剪辑...\n')
        w,h=850,480
        clips=[]
        n=0
        drtn=3
        crstime=1
        for p in self.lst:
            fn=os.path.join(self.pth,p)
            if n==0:
                _img=ImageClip(fn).set_fps(25).set_duration(drtn).resize((w,h))
                clips.append(_img)
                n+=1
            else:
                _img=ImageClip(fn).set_fps(25).set_duration(drtn).resize((w,h)).crossfadein(crstime).set_start((drtn-crstime)*n)
                clips.append(_img)
                n+=1    
                
        #计算正片时间，供logo字幕用
        drt=0
        for c in clips:
            drt=drt+(c.duration-crstime)
        subtitle_drt=drt-clips[0].duration                            
                
        #加上片尾
        print('正在处理片尾...\n')
        endvideo=VideoFileClip(self.endV,target_resolution=(h,w)).set_start((drtn-crstime)*n) # 片尾   分辨率是先写h,再写w  大坑
        clips.append(endvideo)        
        
        print('正在处理字幕...\n')
        text = TextClip(txt="大颗粒积木搭建", fontsize=30, font='Microsoft-YaHei-&-Microsoft-YaHei-UI',color='#95ff67').set_pos((15,15)).set_duration(3)
        clips.append(text)
        text=self.subtitle_right_btm(self.consName[3:],60,w,h)
        clips.append(text)
        text = TextClip(txt='大智小超科学实验室', fontsize=15, \
                 font='Microsoft-YaHei-&-Microsoft-YaHei-UI',color='#9EACC1') \
                .set_pos((685,8)).set_start(clips[0].duration).set_duration(subtitle_drt)        
        clips.append(text)
                
        # finalclip = concatenate_videoclips([_img1,_img2])
        print('正在拼接视频...\n')
        finalclip = CompositeVideoClip(clips)
        
        #加上背景音乐
        print('正在添加背景音乐...\n')
        BGM=AudioFileClip(self.bgm).set_duration(finalclip.duration-endvideo.duration).fx(afx.audio_fadeout,0.8)
        final_audio = CompositeAudioClip([BGM,finalclip.audio])
        mix=finalclip.set_audio(final_audio)
        
        
        out=os.path.join(self.pth,self.consName,self.consName+'_搭建视频.mp4')
        print('正在导出视频：{}...\n'.format(out))
        mix.write_videofile(out)
        print('Done')
        
    def subtitle_right_btm(self,txt,fsz,w,h):
        x=w-len(txt)*fsz-30
        y=h-fsz-30
        text = TextClip(txt=txt, fontsize=fsz, font='Microsoft-YaHei-&-Microsoft-YaHei-UI',color='#95ff67').set_pos((x,y)).set_duration(3)
        return text
    
    
if __name__=='__main__':
#     mylego=LegoCons('I:\\乐高\\图纸','003鳄鱼')
#     mylego.expHTML_lego()
    
    myMovie=ConsMovie('I:\\乐高\\图纸','003鳄鱼')
    myMovie.expMovie()

In [ ]:
import os
from moviepy.editor import *
import moviepy.audio.fx.all as afx
from moviepy.video.tools.subtitles import SubtitlesClip

FONT_URL = 'K:\fonts\zhuangjiamingchao.ttf'


pth='I:\\乐高\\图纸\\002长颈鹿'
p1=os.path.join(pth,'001_1x.png')
p2=os.path.join(pth,'002_1x.png')
p3=os.path.join(pth,'003_1x.png')

endV='I:\大智小超\公共素材\视频类\片尾01.mp4'
sv='I:\大智小超\公共素材\视频类\片头.mp4'



# _img=ImageClip([p1,p2,p3])
# _img.resize((850,480))
# _img.fps=5
# _img.set_duration(3)
# print(_img.duration)

ps=[p1,p2,p3]

clips=[]
w,h=850,480
n=0
drtn=3
crstime=1
for p in ps:
    if n==0:
        _img=ImageClip(p).set_fps(25).set_duration(drtn).resize((w,h)).crossfadein(crstime)
        clips.append(_img)
        n+=1
    else:
        _img=ImageClip(p).set_fps(25).set_duration(drtn).resize((w,h)).crossfadein(crstime).set_start((drtn-crstime)*n)
        clips.append(_img)
        n+=1    
        
# endvideo=VideoFileClip(endV,target_resolution=(h,w)).set_start((drtn-crstime)*n) #分辨率是先写h,再写w  大坑
# clips.append(endvideo)

# subs = [((0, 3), 'sub1'),
#         ((3, 7), 'sub2'),
#         ((9, 11), 'sub3'),
#         ((11, 16), 'sub4')]

# subtitles = SubtitlesClip(subs)
# print ( TextClip.list("font") )

# print(TextClip.list('color'))

drt=0
for c in clips:
    drt=drt+int(c.duration)
    print(c.duration)
print(drt)

text1 = TextClip(txt='大颗粒积木搭建', fontsize=30, font='Microsoft-YaHei-&-Microsoft-YaHei-UI',color='#95ff67').set_pos((15,15)).set_duration(2)
text2 = TextClip(txt='长颈鹿', fontsize=60, font='Microsoft-YaHei-&-Microsoft-YaHei-UI',color='#95ff67').set_pos((640,390)).set_duration(2)
text3 = TextClip(txt='大智小超科学实验室', fontsize=15, \
                 font='Microsoft-YaHei-&-Microsoft-YaHei-UI',color='#9EACC1') \
                .set_pos((685,8)).set_start(clips[0].duration).set_duration(drt-clips[0].duration-clips[-1].duration)

clips.append(text1)
clips.append(text2)
clips.append(text3)

out=os.path.join(pth,'test_video.mp4')
# finalclip = concatenate_videoclips([_img1,_img2])

finalclip = CompositeVideoClip(clips)

# f=CompositeVideoClip([clips])

# BGM=AudioFileClip('I:\大智小超\公共素材\声音类\legoBGM.mp3').set_duration(3).fx(afx.audio_fadeout,0.5)
# final_audio = CompositeAudioClip([BGM,finalclip.audio])
# mix=finalclip.set_audio(final_audio)





finalclip.write_videofile(out)


